On commence par importer les bibliothèques nécessaires. Attention : NE PAS EFFACER !

In [ ]:
from pylab import *    # charge toutes les bibliothèques scientifiques
from scipy.optimize import curve_fit    # permet de modéliser selon un modèle personnalisé
from lmfit import minimize,Parameters,Parameter,report_fit
from lmfit.models import ExpressionModel

from time import *    # pour les mesures de temps

# On initialise la carte Arduino avec le protocole firmata
import pyfirmata
carte = pyfirmata.Arduino('/dev/ttyACM0')
print("Connexion établie")
acquisition = pyfirmata.util.Iterator(carte)
acquisition.start()

# Paramètres de graphique
%matplotlib ipympl
rcParams['figure.figsize'] = [16, 8]    # pour une fenetre de graphique de taille adaptée
rcParams['font.size'] = 15    # pour un texte plus grand
rcParams['lines.markersize'] = 15    # pour des symboles plus grands
rcParams['lines.markeredgewidth'] = 2    # pour des symboles plus larges

Définition des voies d'entrée / sortie :

In [ ]:
EA0 = carte.get_pin('a:0:i')
DS13 = carte.get_pin('d:13:o')

Interaction avec les voies définies précédemment :

In [ ]:
while EA0.read() == None:
    pass    # Tant que la mesure analogique ne retourne pas de valeur, on ne commence pas

Calcul des caractéristiques de la courbe de régression. Attention : NE PAS SUPPRIMER, et ne le modifier que si les paramètres par défaut ne donnent pas de bon résultat :

In [ ]:
def Modele(expression, x, y):
    
    modele = ExpressionModel(expression)    # si la variable indépendante n'est pas "x", ajouter comme argument independant_vars = ["nom"])

    # Cette partie devine les paramètres et leur attribue une valeur initiale de 1 pour la recherche
    parametres = modele.make_params()
    for i in parametres :
        modele.set_param_hint(i, value = 1)
    parametres = modele.make_params()

    #modele.set_param_hint('a', value = 1, vary = True, min = -inf, max = inf)    # À compléter pour changer les valeurs initiales des paramètres. Le dernier argument est expr (string)

    resultat = modele.fit(y, parametres, x = x)    # Entrer ici le nom de la variable en x et en y, pour le calcul final des paramètres

    # On affiche les valeurs trouvées pour les paramètres :
    valeurs = ""
    for key in resultat.params:
        valeurs += f"{key} = {resultat.params[key].value} ; incertitude : {resultat.params[key].stderr}\n"
    return(modele, resultat.params, valeurs, expression)

Tracer de graphique(s) à partir des tableaux précédents :

In [ ]:
subplot(1, 1, 1)    # lignes, colonnes, numéro du graphique

# Tracé des points expérimentaux :
plot(x, y, '+', label="")    # abscisse, ordonnée, options : + o . , s points   - -- -. : lignes  b g r y m c k w couleurs ; label est la légende associée à la courbe

# Tracé du modèle :
xMod = linspace(min(x), max(x), 30)    # Intervalle de valeurs pour le modèle : (début, fin, nombre de points)

modele, parametres, valeurs, expression = Modele("a * x + b", x, y)    # Remplir ici l'expression du modèle, et les variables en abscisse et en ordonnée
yMod = modele.eval(parametres, x = xMod)
plot(xMod, yMod, '-', label=f"Modèle : {expression} \n {valeurs}" )    # trace la courbe du modèle


#axis('equal')    # à décommenter si on veut un repère orthonormé
grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Mon titre")    # mettre le titre désiré entre les guillemets
xlabel("")    # mettre le nom de l'axe des abscisses entre les guillemets
ylabel("");    # mettre le nom de l'axe des ordonnées entre les guillemets. Le point-virgule final évite d'afficher un texte récapitulatif.